## Consultas API SDOT Street Parking Category

In [1]:
import os
import urllib.request

def dl_data(url, output):
    try:        
        f = urllib.request.urlopen(url)
        print ("Downloading " + url)

        os.makedirs(os.path.dirname(output), exist_ok=True)
        with open(output, "wb") as local_file:
            local_file.write(f.read())

    except URLError:
        print ("Error", url)

In [2]:
# Url 1000 primeros ids:
url1 = 'http://gisrevprxy.seattle.gov/arcgis/rest/services/SDOT_EXT/DSG_datasharing/MapServer/14/query?where=1%3D1&text=&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&relationParam=&outFields=*&returnGeometry=false&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&returnDistinctValues=false&resultOffset=&resultRecordCount=&queryByDistance=&returnExtentsOnly=false&datumTransformation=&parameterValues=&rangeValues=&f=pjson'

In [3]:
output1 = './data/street_parking_ids_1_1000.json'
dl_data(url1, output1)

In [4]:
# Url 1000 siguientes ids (Result Offset = 1000):
url2 = 'http://gisrevprxy.seattle.gov/arcgis/rest/services/SDOT_EXT/DSG_datasharing/MapServer/14/query?where=1%3D1&text=&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&relationParam=&outFields=*&returnGeometry=false&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&returnDistinctValues=false&resultOffset=1000&resultRecordCount=&queryByDistance=&returnExtentsOnly=false&datumTransformation=&parameterValues=&rangeValues=&f=pjson'

## Consolidación de respuesta en fichero csv

In [5]:
import json
import codecs
import pandas as pd

In [6]:
data1 = json.load(codecs.open(output1, 'r', 'utf-8-sig'))

ek1 = [data1['features'][i]['attributes']['ELMNTKEY'] for i in range(len(data1['features']))]
ts1 = [data1['features'][i]['attributes']['TOTAL_SPACES'] for i in range(len(data1['features']))]
tnp1 = [data1['features'][i]['attributes']['TOTAL_NOPARK'] for i in range(len(data1['features']))]
tz1 = [data1['features'][i]['attributes']['TOTAL_ZONES'] for i in range(len(data1['features']))]
ps1 = [data1['features'][i]['attributes']['PARKING_SPACES'] for i in range(len(data1['features']))]
pc1 = [data1['features'][i]['attributes']['PARKING_CATEGORY'] for i in range(len(data1['features']))]

d1 = {'element_key': ek1, 'total_spaces': ts1, 'total_nopark': tnp1, 'total_zones': tz1, 'parking_spaces': ps1,
     'parking_category': pc1}
df = pd.DataFrame(data = d1, dtype='int32')
df = df.dropna()

In [7]:
urlI = 'http://gisrevprxy.seattle.gov/arcgis/rest/services/SDOT_EXT/DSG_datasharing/MapServer/14/query?where=1%3D1&text=&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&relationParam=&outFields=*&returnGeometry=false&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&returnDistinctValues=false&resultOffset='
urlE = '&resultRecordCount=&queryByDistance=&returnExtentsOnly=false&datumTransformation=&parameterValues=&rangeValues=&f=pjson'
for i in range(1,50):
    url = urlI + str(i*1000) + urlE
    output = './data/street_parking_ids.json'
    dl_data(url, output)
    data1 = json.load(codecs.open(output, 'r', 'utf-8-sig'))
    ek1 = [data1['features'][i]['attributes']['ELMNTKEY'] for i in range(len(data1['features']))]
    ts1 = [data1['features'][i]['attributes']['TOTAL_SPACES'] for i in range(len(data1['features']))]
    tnp1 = [data1['features'][i]['attributes']['TOTAL_NOPARK'] for i in range(len(data1['features']))]
    tz1 = [data1['features'][i]['attributes']['TOTAL_ZONES'] for i in range(len(data1['features']))]
    ps1 = [data1['features'][i]['attributes']['PARKING_SPACES'] for i in range(len(data1['features']))]
    pc1 = [data1['features'][i]['attributes']['PARKING_CATEGORY'] for i in range(len(data1['features']))]
    d1 = {'element_key': ek1, 'total_spaces': ts1, 'total_nopark': tnp1, 'total_zones': tz1, 'parking_spaces': ps1,
     'parking_category': pc1}
    df1 = pd.DataFrame(data = d1, dtype='int32')
    df1 = df1.dropna()
    df = pd.concat([df, df1], axis=0, sort=True)

In [9]:
df.element_key.duplicated().sum()

0

In [10]:
df.isnull().sum()

element_key         0
parking_category    0
parking_spaces      0
total_nopark        0
total_spaces        0
total_zones         0
dtype: int64

In [11]:
df.shape

(46101, 6)

In [12]:
df.head()

,element_key,parking_category,parking_spaces,total_nopark,total_spaces,total_zones
0,32018,Paid Parking,6,8,15,1
1,1042,No Parking Allowed,0,5,6,1
2,48129,Paid Parking,4,6,11,1
3,47962,Paid Parking,7,8,17,2
4,47966,Paid Parking,4,5,10,1


In [13]:
df[['element_key','parking_category','total_nopark','total_zones','parking_spaces','total_spaces']].to_csv(
    './data/StreetParking.csv', index=False)